### Explain Positional Encoding

In the previous two notebooks we got to know to important jargon related to training\
language models. We got to know embedding, vocabs, masking, ...

In this notebook we will got to know "positional encoding". Early language models used\
Recurrent Neural Networks (RNNs) as backbones. In such models tokens are fed one after\
one in sequential manner. The dependency of one word over the previouse word was implicit.

In transformers however as we saw in the masking notebook. We fed a sequence of words/tokens\
all at once. we know that words positions matters. The "Why transformers are better than LSTM"\
is different from "Why LSTM are better than transformers". The order matters.

To fix the transformers missing positional information the authors of trasnformers deliberately\
added positional information. consider this:

["Being", "strong", "is", "all", "what", "matters"]

To add the positional information we simply add the order. Like this.

[1, 2, 3, 4, 5, 6]

Wrong!



### Position as vectors

Neural networks operates on real valued vectors. We can't simply add poistion as an integer\
in the input. For back-propagation to work and for higher layers ingesting positional information,\
position should be represented as a constant vector alongside the embedding.

In mathmatical terms each token now is represented as following:
$$ token = P_{token} + E_{token}$$

Where the dimension of each vector P (position), E (Embedding) are of equal sizes.\
In exact representation of $E_{token}$ is $(e_0, e_1, e_2)$, $P_{token}$ is $(p_0, p_1, p_2)$\
It is important to understand that $P$ vector is constant among all sentences.\
We compute it once during model initialization and we use it with each entry. 

### Computing Positional Encoding
So we need a mathmatical trick to create for each position in a sequence a vector\
of a sepcific size and certainly the vectors shouldn't be similar for a given position.\
The following part is not important you may skip it.

The authors of the transformers introduced the following mathmatical trick to generate\
the positional vectors for input sequence:

\begin{align}
  PE_{(pos, 2i)} = sin(pos/10000^{2i/d_{model}}) \\
  PE_{(pos, 2i+1)} = cos(pos/10000^{2i/d_{model}})
\end{align}

How intimidating.

We are transforming each position into a fixed vector. Assume that we set the embedding\
and positional encoding dimensionality into 30 and our sequence size is 10. Then each\
position from 1 to 10 now is represented as following:

    1 ->  [a0, a1, a2, a3, a4, a5, a6, ..., a30]
    2 ->  [b0, b1, b2, b3, b4, b5, b6, ..., b30]
    .
    10 -> [c0, c1, c2, c3, c4, c5, c6, ..., c30]

The values a0-a30 and b0-b30 are computed as following:

    a0 = sin( "1"/10000**(2"0"/"30") ) (even index 0 use sin)
    a1 = cos( "1"/10000**(2"1"/"30") ) (odd index 1 use cos)
    a2 = sin( "1"/10000**(2"2"/"30") ) (even index 2 use sin)
    ...

The application of these trignometric functions gurantees differnet vectors\
for each position. No matter the dimension size. Ensuring the values of all\
dimensions are less than one.
In the below code we have a simple implemenation.


In [3]:
import numpy as np

def positional_encoding(pos, i, dim):
    if i%2==0:
        return np.sin(pos/10000**(2*i/dim))
    else:
        return np.cos(pos/10000**(2*i/dim))

dim = 3
pos_1_vec = [positional_encoding(1, i, dim) for i in range(dim)]
pos_1_vec

[0.8414709848078965, 0.9999976792064809, 4.641588833596115e-06]

In practice we can simply use pytorch's ``nn.embedding``.